In [1]:
import pandas as pd
from math import sqrt
import datetime
import numpy as np
import re

In [68]:
usersDF = pd.read_csv('../../../Downloads/myanimelist/users_cleaned.csv')
animesDF = pd.read_csv('../../../Downloads/myanimelist/animesDF.csv')
scoresDF = pd.read_csv('../../../Downloads/myanimelist/scores.csv')

In [75]:
print(usersDF.shape, animesDF.shape, scoresDF.shape)

(108711, 17) (2084, 33) (608131, 4)


In [70]:
anime_ids = np.sort(animesDF.anime_id.unique())
len(anime_ids)

2084

In [71]:
top_1000_users = list(usersDF.sort_values(by=['user_completed'], ascending=False).head(1000)['username'])

In [72]:
#scoresDF = scoresDF.sample(n=10000, random_state=15)
#scoresDF.to_csv('../../../Downloads/myanimelist/scores_sample.csv', index=False)

In [73]:
scoresDF = scoresDF[scoresDF['username'].isin(top_1000_users)]

In [74]:
scoresDF = scoresDF[scoresDF['anime_id'].isin(anime_ids)]

In [53]:
animesDF = animesDF[(animesDF['duration_min']>=20) \
                    & (animesDF['aired_from_year']>=2000) \
                    & ~(animesDF.related.str.contains('Prequel')) \
                    & (animesDF['type'] != 'Special') \
                    & ~(animesDF.genre.str.contains('Kids')) \
                    & ~(animesDF.genre.str.contains('Shoujo Ai')) \
                    & ~(animesDF.genre.str.contains('Shounen Ai')) \
                    & ~(animesDF.genre.str.contains('Yaoi')) \
                    & ~(animesDF.genre.str.contains('Yuri'))]
animesDF.to_csv('../../../Downloads/myanimelist/animesDF.csv', index=False)

In [77]:
animesDF.sort_values(by=['anime_id']).head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
1622,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ天国の扉,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,NaN,"{'Parent story': [{'mal_id': 1, 'type': 'anime...","Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"Action, Space, Drama, Mystery, Sci-Fi","['""Ask DNA"" by The Seatbelts featuring Raju Ra...","['""Gotta Knock a Little Harder"" by The Seatbel...",114.0,2001.0
1490,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN,WHR,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,Tuesdays at Unknown,[],Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Magic, Police, Supernatural, Drama, My...","['""Shell"" by Bana']","['#1: ""Half Pain"" by Bana (eps 01-14,16-25)', ...",25.0,2002.0
1366,8,Beet the Vandel Buster,Beet the Vandel Buster,冒険王ビィト,"Adventure King Beet, Bouken Ou Beet",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,52,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",NaN,NaN,Toei Animation,"Adventure, Fantasy, Shounen, Supernatural","['#1: ""Emotion"" by Sunbrain', '#2: ""Wish Men"" ...","['#1: ""pureness"" by Nana Kitade (eps 1-13)', '...",23.0,2004.0
885,15,Eyeshield 21,NaN,アイシールド21,Eyeshield21,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,145,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 43, 'type': 'manga'...","TV Tokyo, Nihon Ad Systems, TV Tokyo Music, Sh...",Sentai Filmworks,Studio Gallop,"Action, Sports, Comedy, Shounen","['#1: ""Breakthrough"" by Coming Century (eps 1-...","['#1: ""Be Free"" by Rikken&#039;z (eps 1-13)', ...",23.0,2005.0
214,16,Hachimitsu to Clover,Honey and Clover,ハチミツとクローバー,"HachiKuro, Honey & Clover",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,24,Finished Airing,...,Fridays at 00:35 (JST),"{'Adaptation': [{'mal_id': 1009, 'type': 'mang...","Genco, Fuji TV, Shueisha",Viz Media,J.C.Staff,"Comedy, Drama, Josei, Romance, Slice of Life","['""Dramatic"" by Yuki']","['#1: ""Waltz"" by Suneohair (eps 1-12, 24)', '#...",23.0,2005.0


In [78]:
scoresDF.head()

,username,anime_id,my_score,my_status
866,MistButterfly,59,6,2
867,MistButterfly,74,7,2
868,MistButterfly,120,8,2
869,MistButterfly,178,5,2
872,MistButterfly,249,7,2


In [79]:
anime_users = scoresDF.pivot(index='anime_id', columns='username', values='my_score')

In [63]:
'''
gets all user scores
'''
def get_user_scores(scoresDF, viewer):
    return scoresDF.loc[(scoresDF['username'] == viewer)]

get_user_scores(scoresDF, 'MistButterfly').head()

,username,anime_id,my_score,my_status
866,MistButterfly,59,6,2
867,MistButterfly,74,7,2
868,MistButterfly,120,8,2
869,MistButterfly,178,5,2
872,MistButterfly,249,7,2


In [64]:
# rewrite all with new dataframe, speed increase expected
def sim_pearson(anime_users, anime1_id, anime2_id):
    '''Return the Pearson correlation coefficient for anime1 and anime2.'''
    both_scores = anime_users.loc[[anime1_id, anime2_id],:].dropna(axis='columns').to_numpy()
    scores_anime1 = both_scores[0]
    scores_anime2 = both_scores[1]
    
    shared_viewers = len(scores_anime1)
    
    # if no ratings in common, return 0
    if shared_viewers == 0:
        return 0

    # add up all the preferences
    sum1 = np.sum(scores_anime1)
    sum2 = np.sum(scores_anime2)
    
    # sum up the squares
    sum1Sq = np.sum(scores_anime1**2)
    sum2Sq = np.sum(scores_anime2**2)
    
    # sum up the products
    #pSum = sum([score1*score2 for score1, score2 in zip(scores1, scores2)])
    pSum = np.dot(scores_anime1, scores_anime2)
    # calculate Pearson score
    num = pSum - (sum1 * sum2)/shared_viewers
    den = sqrt((sum1Sq - pow(sum1, 2)/shared_viewers) * (sum2Sq - pow(sum2, 2)/shared_viewers))
    if den == 0:
        return 0
    else:
        return round(num/den, 10)

In [65]:
sim_pearson(anime_users, 31964, 20)

0.3753254921

In [66]:
'''
builds table of similarity of the shows
a[i][j] - similarity of the shows with ids: anime_ids[i] and anime_ids[j]
'''
def get_similarity_table(scores, anime_ids):
    similarity_table = np.zeros((len(anime_ids), len(anime_ids)))
    n = len(anime_ids)
    counter1 = 0
    counter2 = 0
    for anime1_id in anime_ids[:1]:
        counter2 = 0
        for anime2_id in anime_ids:
            if anime1_id != anime2_id:
                similarity = sim_pearson(scores, anime1_id, anime2_id)
                similarity_table[counter1][counter2] = similarity
                similarity_table[counter2][counter1] = similarity
                counter2 += 1
                print('Processed {} of {} animes :: {}'.format(counter2, n, datetime.datetime.now()))
        np.save('similarity_table', similarity_table)
        counter1 += 1
    return similarity_table

In [67]:
print(datetime.datetime.now())
get_similarity_table(anime_users, anime_ids)

2020-02-02 21:29:03.679298
Processed 1 of 2084 animes :: 2020-02-02 21:29:03.686734
Processed 2 of 2084 animes :: 2020-02-02 21:29:03.691410
Processed 3 of 2084 animes :: 2020-02-02 21:29:03.695678
Processed 4 of 2084 animes :: 2020-02-02 21:29:03.699811
Processed 5 of 2084 animes :: 2020-02-02 21:29:03.704805
Processed 6 of 2084 animes :: 2020-02-02 21:29:03.708921
Processed 7 of 2084 animes :: 2020-02-02 21:29:03.713952
Processed 8 of 2084 animes :: 2020-02-02 21:29:03.719552
Processed 9 of 2084 animes :: 2020-02-02 21:29:03.723960
Processed 10 of 2084 animes :: 2020-02-02 21:29:03.727264
Processed 11 of 2084 animes :: 2020-02-02 21:29:03.730674
Processed 12 of 2084 animes :: 2020-02-02 21:29:03.736666
Processed 13 of 2084 animes :: 2020-02-02 21:29:03.741800
Processed 14 of 2084 animes :: 2020-02-02 21:29:03.745318
Processed 15 of 2084 animes :: 2020-02-02 21:29:03.749722
Processed 16 of 2084 animes :: 2020-02-02 21:29:03.753938
Processed 17 of 2084 animes :: 2020-02-02 21:29:03.759

Processed 155 of 2084 animes :: 2020-02-02 21:29:04.294960
Processed 156 of 2084 animes :: 2020-02-02 21:29:04.298728
Processed 157 of 2084 animes :: 2020-02-02 21:29:04.303513
Processed 158 of 2084 animes :: 2020-02-02 21:29:04.307225
Processed 159 of 2084 animes :: 2020-02-02 21:29:04.311078
Processed 160 of 2084 animes :: 2020-02-02 21:29:04.315906
Processed 161 of 2084 animes :: 2020-02-02 21:29:04.320881
Processed 162 of 2084 animes :: 2020-02-02 21:29:04.324484
Processed 163 of 2084 animes :: 2020-02-02 21:29:04.327847
Processed 164 of 2084 animes :: 2020-02-02 21:29:04.331467
Processed 165 of 2084 animes :: 2020-02-02 21:29:04.335909
Processed 166 of 2084 animes :: 2020-02-02 21:29:04.340263
Processed 167 of 2084 animes :: 2020-02-02 21:29:04.343927
Processed 168 of 2084 animes :: 2020-02-02 21:29:04.347518
Processed 169 of 2084 animes :: 2020-02-02 21:29:04.351196
Processed 170 of 2084 animes :: 2020-02-02 21:29:04.354529
Processed 171 of 2084 animes :: 2020-02-02 21:29:04.3580

Processed 316 of 2084 animes :: 2020-02-02 21:29:04.913468
Processed 317 of 2084 animes :: 2020-02-02 21:29:04.917396
Processed 318 of 2084 animes :: 2020-02-02 21:29:04.924960
Processed 319 of 2084 animes :: 2020-02-02 21:29:04.933185
Processed 320 of 2084 animes :: 2020-02-02 21:29:04.940487
Processed 321 of 2084 animes :: 2020-02-02 21:29:04.945189
Processed 322 of 2084 animes :: 2020-02-02 21:29:04.948690
Processed 323 of 2084 animes :: 2020-02-02 21:29:04.952281
Processed 324 of 2084 animes :: 2020-02-02 21:29:04.957651
Processed 325 of 2084 animes :: 2020-02-02 21:29:04.961503
Processed 326 of 2084 animes :: 2020-02-02 21:29:04.965080
Processed 327 of 2084 animes :: 2020-02-02 21:29:04.968849
Processed 328 of 2084 animes :: 2020-02-02 21:29:04.972661
Processed 329 of 2084 animes :: 2020-02-02 21:29:04.976003
Processed 330 of 2084 animes :: 2020-02-02 21:29:04.979596
Processed 331 of 2084 animes :: 2020-02-02 21:29:04.983189
Processed 332 of 2084 animes :: 2020-02-02 21:29:04.9869

Processed 488 of 2084 animes :: 2020-02-02 21:29:05.754088
Processed 489 of 2084 animes :: 2020-02-02 21:29:05.761782
Processed 490 of 2084 animes :: 2020-02-02 21:29:05.765464
Processed 491 of 2084 animes :: 2020-02-02 21:29:05.769255
Processed 492 of 2084 animes :: 2020-02-02 21:29:05.773197
Processed 493 of 2084 animes :: 2020-02-02 21:29:05.778964
Processed 494 of 2084 animes :: 2020-02-02 21:29:05.783487
Processed 495 of 2084 animes :: 2020-02-02 21:29:05.787210
Processed 496 of 2084 animes :: 2020-02-02 21:29:05.791019
Processed 497 of 2084 animes :: 2020-02-02 21:29:05.797553
Processed 498 of 2084 animes :: 2020-02-02 21:29:05.802327
Processed 499 of 2084 animes :: 2020-02-02 21:29:05.806642
Processed 500 of 2084 animes :: 2020-02-02 21:29:05.813325
Processed 501 of 2084 animes :: 2020-02-02 21:29:05.818215
Processed 502 of 2084 animes :: 2020-02-02 21:29:05.822726
Processed 503 of 2084 animes :: 2020-02-02 21:29:05.828338
Processed 504 of 2084 animes :: 2020-02-02 21:29:05.8334

Processed 657 of 2084 animes :: 2020-02-02 21:29:06.582243
Processed 658 of 2084 animes :: 2020-02-02 21:29:06.588606
Processed 659 of 2084 animes :: 2020-02-02 21:29:06.594275
Processed 660 of 2084 animes :: 2020-02-02 21:29:06.600061
Processed 661 of 2084 animes :: 2020-02-02 21:29:06.604096
Processed 662 of 2084 animes :: 2020-02-02 21:29:06.609284
Processed 663 of 2084 animes :: 2020-02-02 21:29:06.616039
Processed 664 of 2084 animes :: 2020-02-02 21:29:06.621297
Processed 665 of 2084 animes :: 2020-02-02 21:29:06.626509
Processed 666 of 2084 animes :: 2020-02-02 21:29:06.630713
Processed 667 of 2084 animes :: 2020-02-02 21:29:06.636917
Processed 668 of 2084 animes :: 2020-02-02 21:29:06.642421
Processed 669 of 2084 animes :: 2020-02-02 21:29:06.646826
Processed 670 of 2084 animes :: 2020-02-02 21:29:06.653237
Processed 671 of 2084 animes :: 2020-02-02 21:29:06.658388
Processed 672 of 2084 animes :: 2020-02-02 21:29:06.663382
Processed 673 of 2084 animes :: 2020-02-02 21:29:06.6688

...

Processed 1476 of 2084 animes :: 2020-02-02 21:29:10.521903
Processed 1477 of 2084 animes :: 2020-02-02 21:29:10.530268
Processed 1478 of 2084 animes :: 2020-02-02 21:29:10.535635
Processed 1479 of 2084 animes :: 2020-02-02 21:29:10.539777
Processed 1480 of 2084 animes :: 2020-02-02 21:29:10.543634
Processed 1481 of 2084 animes :: 2020-02-02 21:29:10.550285
Processed 1482 of 2084 animes :: 2020-02-02 21:29:10.556363
Processed 1483 of 2084 animes :: 2020-02-02 21:29:10.560999
Processed 1484 of 2084 animes :: 2020-02-02 21:29:10.567346
Processed 1485 of 2084 animes :: 2020-02-02 21:29:10.573499
Processed 1486 of 2084 animes :: 2020-02-02 21:29:10.577926
Processed 1487 of 2084 animes :: 2020-02-02 21:29:10.583130
Processed 1488 of 2084 animes :: 2020-02-02 21:29:10.588933
Processed 1489 of 2084 animes :: 2020-02-02 21:29:10.593766
Processed 1490 of 2084 animes :: 2020-02-02 21:29:10.597524
Processed 1491 of 2084 animes :: 2020-02-02 21:29:10.604112
Processed 1492 of 2084 animes :: 2020-02

Processed 1630 of 2084 animes :: 2020-02-02 21:29:11.351928
Processed 1631 of 2084 animes :: 2020-02-02 21:29:11.360050
Processed 1632 of 2084 animes :: 2020-02-02 21:29:11.368515
Processed 1633 of 2084 animes :: 2020-02-02 21:29:11.374234
Processed 1634 of 2084 animes :: 2020-02-02 21:29:11.378281
Processed 1635 of 2084 animes :: 2020-02-02 21:29:11.383064
Processed 1636 of 2084 animes :: 2020-02-02 21:29:11.388600
Processed 1637 of 2084 animes :: 2020-02-02 21:29:11.394006
Processed 1638 of 2084 animes :: 2020-02-02 21:29:11.398804
Processed 1639 of 2084 animes :: 2020-02-02 21:29:11.404185
Processed 1640 of 2084 animes :: 2020-02-02 21:29:11.410128
Processed 1641 of 2084 animes :: 2020-02-02 21:29:11.415786
Processed 1642 of 2084 animes :: 2020-02-02 21:29:11.420764
Processed 1643 of 2084 animes :: 2020-02-02 21:29:11.426438
Processed 1644 of 2084 animes :: 2020-02-02 21:29:11.431332
Processed 1645 of 2084 animes :: 2020-02-02 21:29:11.435464
Processed 1646 of 2084 animes :: 2020-02

Processed 1787 of 2084 animes :: 2020-02-02 21:29:12.184225
Processed 1788 of 2084 animes :: 2020-02-02 21:29:12.188971
Processed 1789 of 2084 animes :: 2020-02-02 21:29:12.194491
Processed 1790 of 2084 animes :: 2020-02-02 21:29:12.199014
Processed 1791 of 2084 animes :: 2020-02-02 21:29:12.204784
Processed 1792 of 2084 animes :: 2020-02-02 21:29:12.210214
Processed 1793 of 2084 animes :: 2020-02-02 21:29:12.216397
Processed 1794 of 2084 animes :: 2020-02-02 21:29:12.221678
Processed 1795 of 2084 animes :: 2020-02-02 21:29:12.227578
Processed 1796 of 2084 animes :: 2020-02-02 21:29:12.232576
Processed 1797 of 2084 animes :: 2020-02-02 21:29:12.237932
Processed 1798 of 2084 animes :: 2020-02-02 21:29:12.245262
Processed 1799 of 2084 animes :: 2020-02-02 21:29:12.253028
Processed 1800 of 2084 animes :: 2020-02-02 21:29:12.257616
Processed 1801 of 2084 animes :: 2020-02-02 21:29:12.267109
Processed 1802 of 2084 animes :: 2020-02-02 21:29:12.272316
Processed 1803 of 2084 animes :: 2020-02

Processed 1951 of 2084 animes :: 2020-02-02 21:29:13.007744
Processed 1952 of 2084 animes :: 2020-02-02 21:29:13.012383
Processed 1953 of 2084 animes :: 2020-02-02 21:29:13.017953
Processed 1954 of 2084 animes :: 2020-02-02 21:29:13.021651
Processed 1955 of 2084 animes :: 2020-02-02 21:29:13.026471
Processed 1956 of 2084 animes :: 2020-02-02 21:29:13.032692
Processed 1957 of 2084 animes :: 2020-02-02 21:29:13.037151
Processed 1958 of 2084 animes :: 2020-02-02 21:29:13.042268
Processed 1959 of 2084 animes :: 2020-02-02 21:29:13.048532
Processed 1960 of 2084 animes :: 2020-02-02 21:29:13.053257
Processed 1961 of 2084 animes :: 2020-02-02 21:29:13.058773
Processed 1962 of 2084 animes :: 2020-02-02 21:29:13.065034
Processed 1963 of 2084 animes :: 2020-02-02 21:29:13.070726
Processed 1964 of 2084 animes :: 2020-02-02 21:29:13.075973
Processed 1965 of 2084 animes :: 2020-02-02 21:29:13.082043
Processed 1966 of 2084 animes :: 2020-02-02 21:29:13.087732
Processed 1967 of 2084 animes :: 2020-02

array([[0.41659023, 0.31305247, 0.30909409, ..., 0.26352314, 0.59708752,
        0.        ],
       [0.31305247, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.30909409, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.26352314, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.59708752, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [28]:
'''
loads user's scores from MAL xml file converted to csv
'''
def load_user_list(path):
    userDF = pd.read_csv(path)
    return userDF[['series_animedb_id', 'my_score']][userDF['my_score']!=0]

In [29]:
cmlimmDF = load_user_list('../../../Downloads/animelist_cmlimm.csv')

In [30]:
'''
find user's shows that exist in MAL dataset, as some shows aired after dataset was created
'''
def user_shows_in_db(userDF, animesDF):
    return userDF[userDF[['series_animedb_id']].isin(animesDF.anime_id.unique()).any(1)]

In [31]:
np.array(user_shows_in_db(cmlimmDF, animesDF))

array([[32977,     7],
       [22147,     9],
       [35015,     7],
       [ 2251,    10],
       [ 3901,     8],
       [35965,     7],
       [28805,     7],
       [32664,     6],
       [31964,     8],
       [33486,    10],
       [35262,     5],
       [36456,     9],
       [33929,     8],
       [35459,     8],
       [16592,     6],
       [35849,     8],
       [30346,     7],
       [ 6746,     8],
       [ 9314,     7],
       [18507,     7],
       [22265,     8],
       [26213,     8],
       [19671,     7],
       [35198,     8],
       [33731,     5],
       [ 2890,     7],
       [18229,     7],
       [20583,     9],
       [28891,     9],
       [32935,     9],
       [25303,     7],
       [35806,     7],
       [27989,    10],
       [31988,    10],
       [34204,     7],
       [31665,     7],
       [31096,     7],
       [30415,     8],
       [33241,     6],
       [10357,     9],
       [ 5680,     6],
       [ 9617,     6],
       [ 7791,     6],
       [ 97

In [32]:
'''
finds top matches based on shows similarities
'''
def top_matches(similarity_table, user_show_scores):
    result = np.empty((0,2), int)
    user_shows = user_show_scores[:, 0]
    user_scores = user_show_scores[:, 1]
    to_delete = []
    for show in user_shows:
        y = np.where(similarity_table == show)[0][1]
        to_delete.append(y)
    similarity_table = np.delete(similarity_table, to_delete, 1)
    to_delete = []
    for show in similarity_table[1:, 0]:
        if show not in user_shows:
            x = np.where(similarity_table == show)[0][1]
            to_delete.append(x)
    similarity_table = np.delete(similarity_table, to_delete, 0)
    for column in similarity_table.T[1:]:
        result = np.append(result, [[column[0], np.dot(column[1:], user_scores)/np.sum(column[1:])]], axis=0)
    return result[result[:,1].argsort()][::-1]

In [33]:
def show_names(matches, animesDF):
    for show in matches:
        print(animesDF[animesDF['anime_id']==show[0]]['title'].values[0])